In [25]:
import boto3
import uuid
from datetime import datetime

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
table = dynamodb.Table("feedback")

# Generate a unique ID for our new item
new_feedback_id = str(uuid.uuid4())

# Create new item
new_item = {
    'id': new_feedback_id,
    'feedback_comment': 'Initial feedback comment',
    'submitted_at': datetime.now().isoformat()
}

# Add the new item to the table
print("Adding new item to DynamoDB...")
table.put_item(Item=new_item)

# Retrieve and print the item before update
print("\nBefore update:")
response = table.get_item(Key={'id': new_feedback_id})  # Using 'id' instead of 'feedback_id'
original_item = response.get('Item')
print(original_item)

Adding new item to DynamoDB...

Before update:
{'id': '1cff17b4-ee48-4dc9-a9c9-9abbf3ff618c', 'submitted_at': '2025-03-11T15:09:07.289907', 'feedback_comment': 'Initial feedback comment'}


In [26]:
# Update the item
print("\nUpdating item...")
# Using expression attribute names to avoid reserved keywords
update_expression = "SET "
expression_attribute_values = {}
expression_attribute_names = {}

# Values to update
updated_values = {
    'feedback_comment': 'Updated feedback comment',
    'updated_at': datetime.now().isoformat()
}

# Build update expression with attribute names
for i, (key, value) in enumerate(updated_values.items()):
    attribute_name = f"#attr{i}"
    attribute_value = f":val{i}"
    
    update_expression += f"{attribute_name} = {attribute_value}, "
    expression_attribute_names[attribute_name] = key
    expression_attribute_values[attribute_value] = value

# Remove trailing comma and space
update_expression = update_expression[:-2]

# Use 'id' as the key name
table.update_item(
    Key={'id': new_feedback_id},  # Using 'id' instead of 'feedback_id'
    UpdateExpression=update_expression,
    ExpressionAttributeNames=expression_attribute_names,
    ExpressionAttributeValues=expression_attribute_values
)


Updating item...


{'ResponseMetadata': {'RequestId': 'U4AIEPGIDP15HLN46IQMFPP853VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 11 Mar 2025 15:09:10 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'U4AIEPGIDP15HLN46IQMFPP853VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [27]:
# Retrieve and print the item after update
print("\nAfter update:")
response = table.get_item(Key={'id': new_feedback_id})  # Using 'id' instead of 'feedback_id'
updated_item = response.get('Item')
print(updated_item)


After update:
{'updated_at': '2025-03-11T15:09:08.469227', 'id': '1cff17b4-ee48-4dc9-a9c9-9abbf3ff618c', 'submitted_at': '2025-03-11T15:09:07.289907', 'feedback_comment': 'Updated feedback comment'}


In [28]:
# Scan all items
print("\nScanning all items...")
response = table.scan()
items = response.get('Items', [])

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response.get('Items', []))

print(f"Found {len(items)} items in total")



Scanning all items...
Found 45 items in total


In [29]:
# Uncomment this section if you want to delete all items
"""
# Delete all items
print("\nDeleting all items...")
with table.batch_writer() as batch:
    for item in items:
        batch.delete_item(Key={'id': item['id']})  # Using 'id' instead of 'feedback_id'

print(f"Deleted {len(items)} items")
"""

'\n# Delete all items\nprint("\nDeleting all items...")\nwith table.batch_writer() as batch:\n    for item in items:\n        batch.delete_item(Key={\'id\': item[\'id\']})  # Using \'id\' instead of \'feedback_id\'\n\nprint(f"Deleted {len(items)} items")\n'

---
---

In [30]:
dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
table = dynamodb.Table("feedback")

feedback_id = str(uuid.uuid4())

initial_item = {
    'id': feedback_id,
    'version': 1,
    'feedback_comment': 'Initial feedback comment',
    'submitted_at': datetime.now().isoformat()
}

print("Adding initial item to DynamoDB...")
current_item = table.put_item(Item=initial_item)
current_item 

Adding initial item to DynamoDB...


{'ResponseMetadata': {'RequestId': 'BJR5J0ARAGR0E3PI86750H73NRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 11 Mar 2025 15:09:11 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'BJR5J0ARAGR0E3PI86750H73NRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [31]:
response = table.get_item(Key={'id': feedback_id})
response.get('Item')

{'version': Decimal('1'),
 'id': 'd8f17fe7-326a-469e-83a7-36fb13d463f4',
 'submitted_at': '2025-03-11T15:09:09.303116',
 'feedback_comment': 'Initial feedback comment'}

In [32]:
current_version = current_item.get('version', 0)
new_version = current_version + 1

In [33]:
updated_item = {
    'id': feedback_id,
    'version': int(new_version),
    'feedback_comment': 'Updated feedback comment',
    'submitted_at': current_item.get('submitted_at'),
    'updated_at': datetime.now().isoformat()
}

table.put_item(Item=updated_item)

{'ResponseMetadata': {'RequestId': 'PBQBAEB48L6TM583AONTUF0PEVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 11 Mar 2025 15:09:11 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'PBQBAEB48L6TM583AONTUF0PEVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [34]:
response = table.query(
    KeyConditionExpression=boto3.dynamodb.conditions.Key('id').eq(feedback_id)
)
versions = response.get('Items', [])

In [35]:
versions

[{'updated_at': '2025-03-11T15:09:10.416266',
  'version': Decimal('1'),
  'id': 'd8f17fe7-326a-469e-83a7-36fb13d463f4',
  'submitted_at': None,
  'feedback_comment': 'Updated feedback comment'}]